In [ ]:
!pip install LightAutoML -q

# EDA

In [ ]:
import gc
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

warnings.filterwarnings("ignore")

# Загрузка данных
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

# Базовый анализ
print("\n Размеры датасетов ")
print(f"Train: {train_data.shape}")
print(f"Test: {test_data.shape}")

print("\n Типы данных ")
print(train_data.dtypes)

print("\n Пропущенные значения ")
print(train_data.isnull().sum())

print("\n Статистики числовых признаков")
print(train_data.describe())

In [ ]:
# Анализ целевой переменной
print("\n Распределение целевой переменной")
print(train_data["loan_paid_back"].value_counts(normalize=True))

plt.figure(figsize=(6, 4))
train_data["loan_paid_back"].value_counts().plot(kind="bar")
plt.title("Распределение целевой переменной")
plt.xlabel("loan_paid_back")
plt.ylabel("Количество")
plt.show()

In [ ]:
# Определяем признаки
numeric_cols = [
    "annual_income",
    "debt_to_income_ratio",
    "credit_score",
    "loan_amount",
    "interest_rate",
]
categorical_cols = [
    "gender",
    "marital_status",
    "education_level",
    "employment_status",
    "loan_purpose",
    "grade_subgrade",
]

# Распределения числовых признаков
print("\n Анализ числовых признаков ")
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()
for idx, col in enumerate(numeric_cols):
    axes[idx].hist(train_data[col], bins=50, edgecolor="black")
    axes[idx].set_title(f"Распределение {col}")
    axes[idx].set_xlabel(col)
plt.tight_layout()
plt.show()

# Корреляция с целевой переменной
print("\n Корреляция числовых признаков с целевой ")
correlations = (
    train_data[numeric_cols + ["loan_paid_back"]]
    .corr()["loan_paid_back"]
    .sort_values(ascending=False)
)
print(correlations)

# Тепловая карта корреляций
plt.figure(figsize=(10, 8))
sns.heatmap(
    train_data[numeric_cols + ["loan_paid_back"]].corr(), annot=True, cmap="coolwarm", center=0
)
plt.title("Корреляционная матрица")
plt.show()

## Выводы
   - debt_to_income_ratio: -0.336 (сильная отрицательная)
   - credit_score: 0.235 (умеренная положительная)
   - annual_income и loan_amount: слабая корреляция

In [ ]:
# Анализ категориальных признаков
print("\n Анализ категориальных признаков ")
for col in categorical_cols:
    print(f"\n--- {col} ---")
    print(train_data[col].value_counts())

    print(f"\nСредняя вероятность возврата кредита по {col}:")
    probs = train_data.groupby(col)["loan_paid_back"].mean().sort_values(ascending=False)
    print(probs)

# Визуализация категориальных признаков
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.ravel()
for idx, col in enumerate(categorical_cols):
    pd.crosstab(train_data[col], train_data["loan_paid_back"], normalize="index").plot(
        kind="bar", ax=axes[idx]
    )
    axes[idx].set_title(f"{col} vs loan_paid_back")
    axes[idx].set_xlabel(col)
    axes[idx].set_ylabel("Доля")
    axes[idx].legend(title="loan_paid_back")
plt.tight_layout()
plt.show()

## Ключевые находки из EDA

1. **`employment_status` — самый сильный признак**
   - Retired: **99.7%**
   - Unemployed: **7.8%**
Огромная разница в вероятности возврата

2. **`grade_subgrade` — чёткая градация внутри буквенных классов**
   - Пример: A1/A5 (**95.2%/94.5%**)
   - Внутри каждой буквы есть выраженный паттерн по цифре.

3. **Числовые признаки слабокоррелированы**
   - Значит, модель выигрывает от *взаимодействий* (feature crosses).

4. **`education_level`**
   - PhD: **83%**
   - Bachelor’s: **78.9%**
Признак информативный, но не доминирующий.

5. **`loan_purpose`**
   - Home: **82.3%**
   - Education: **77.7%**
Существенные различия между категориями.


In [ ]:
# Анализ выбросов
print("\n Выбросы (IQR метод) ")
for col in numeric_cols:
    Q1 = train_data[col].quantile(0.25)
    Q3 = train_data[col].quantile(0.75)
    IQR = Q3 - Q1
    outliers = ((train_data[col] < (Q1 - 1.5 * IQR)) | (train_data[col] > (Q3 + 1.5 * IQR))).sum()
    print(f"{col}: {outliers} выбросов ({outliers/len(train_data)*100:.2f}%)")

fig, axes = plt.subplots(1, 5, figsize=(20, 4))
for idx, col in enumerate(numeric_cols):
    train_data.boxplot(column=col, by="loan_paid_back", ax=axes[idx])
    axes[idx].set_title(f"{col} по loan_paid_back")
plt.tight_layout()
plt.show()

# FEATURE ENGINEERING

## Обоснования для создаваемых признаков


### 1. Извлечение `grade_digit` из `grade_subgrade`
- Внутри каждой буквенной категории (A-F) есть выраженная градация по цифре.
- Пример: A1/A5 соответствует снижению вероятности возврата (**95.2%/94.5%**).
- Цифра несёт собственную информативность и должна использоваться отдельно.


### 2. Взаимодействия числовых признаков
Корреляции между числовыми признаками низкие, поэтому отдельные фичи малоинформативны.  
Комбинации улучшают модель за счёт доменных зависимостей.

- **income_to_loan_ratio** - способность заёмщика погасить займ.  
- **loan_to_income_ratio** - относительный размер кредита.  
- **debt_times_rate** - комплексная долговая нагрузка (долги x ставка).  
- **income_times_credit** - интегральная «надёжность» (доход x кредитный рейтинг).  
- **monthly_payment_estimate / payment_to_income** - реальная месячная нагрузка.


### 3. Усиление `employment_status` (ключевой признак)
- Разница между категориями экстремально высока:  
  - Retired: **99.7%**  
  - Unemployed: **7.8%**  
- Создаются статистики по числовым признакам внутри каждой категории занятости:
  - Среднее  
  - Отклонение  
  - Процент отклонения  
Это позволяет выделить финансовые профили для каждой группы занятости.


### 4. Групповая статистика по категориальным признакам
Используется стандартная техника для извлечения скрытых паттернов внутри категорий.

**По `grade_subgrade`:** сильная связь (60–95%)  
- mean  
- std  
- разница от среднего  

**По `loan_purpose`:** (77–82%)  
- mean и разница для ключевых числовых признаков  

**По `education_level`:** (78–83%)  
- mean для дохода и кредитного рейтинга  


### 5. Комбинации категорий
Создаются взаимодействия между важными категориальными переменными.

- `employment_status + education_level`  
- `employment_status + marital_status`  
- `grade_subgrade + loan_purpose`  
- `education_level + loan_purpose`  

Цель - уловить различия внутри комбинаций (например, «Retired + PhD» не то же самое, что «Retired + High School»).

### 6. Бинарные флаги
- **high_credit_score ≥ 720** - высокий кредитный рейтинг.  
- **low_debt_ratio ≤ 0.1** - низкая долговая нагрузка.  
- **high_income ≥ 60,000** - устойчивый доход.  
- **small_loan ≤ 10,000** - низкий размер кредита.  
- **low_interest ≤ 11%** - выгодная ставка.

Флаги помогают модели выделять важные пороговые состояния.



In [ ]:
# Перезагружаем данные для FE
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

# Сохраняем ID и target
test_ids = test["id"].copy()
target = train["loan_paid_back"].copy()

# Удаляем служебные колонки
train = train.drop(columns=["id", "loan_paid_back"])
test = test.drop(columns=["id"])

NUMS = ["annual_income", "debt_to_income_ratio", "credit_score", "loan_amount", "interest_rate"]
CATS = [
    "gender",
    "marital_status",
    "education_level",
    "employment_status",
    "loan_purpose",
    "grade_subgrade",
]

print("\nСоздаём признаки на основе инсайтов из EDA")

# ПРИЗНАК 1: grade_digit
print("\nПРИЗНАК 1: Извлечение цифры из grade_subgrade")
train["grade_digit"] = train["grade_subgrade"].str[1].astype("int8")
test["grade_digit"] = test["grade_subgrade"].str[1].astype("int8")

train["grade_letter"] = train["grade_subgrade"].str[0]
test["grade_letter"] = test["grade_subgrade"].str[0]

# ПРИЗНАК 2: Числовые взаимодействия
print("\nПРИЗНАК 2: Взаимодействия числовых признаков")

# Платёжеспособность
train["income_to_loan_ratio"] = train["annual_income"] / (train["loan_amount"] + 1)
test["income_to_loan_ratio"] = test["annual_income"] / (test["loan_amount"] + 1)
print("- income_to_loan_ratio: способность погасить займ")

# Долговая нагрузка
train["loan_to_income_ratio"] = train["loan_amount"] / (train["annual_income"] + 1)
test["loan_to_income_ratio"] = test["loan_amount"] / (test["annual_income"] + 1)
print("- loan_to_income_ratio: размер займа относительно дохода")

# Совокупный риск
train["debt_times_rate"] = train["debt_to_income_ratio"] * train["interest_rate"]
test["debt_times_rate"] = test["debt_to_income_ratio"] * test["interest_rate"]
print("- debt_times_rate: комплексная оценка долговой нагрузки")

# Кредитоспособность
train["income_times_credit"] = train["annual_income"] * train["credit_score"] / 1000000
test["income_times_credit"] = test["annual_income"] * test["credit_score"] / 1000000
print("- income_times_credit: общая финансовая надёжность")

# Месячный платёж и нагрузка
train["monthly_payment_estimate"] = (train["loan_amount"] * train["interest_rate"] / 100) / 12
test["monthly_payment_estimate"] = (test["loan_amount"] * test["interest_rate"] / 100) / 12

train["payment_to_income"] = train["monthly_payment_estimate"] / (train["annual_income"] / 12 + 1)
test["payment_to_income"] = test["monthly_payment_estimate"] / (test["annual_income"] / 12 + 1)
print("- payment_to_income: реальная месячная нагрузка на бюджет")

# ПРИЗНАК 3: employment_status
print("\nПРИЗНАК 3: Фокус на employment_status")

# Групповая статистика по employment
for num_col in NUMS:
    group_mean = train.groupby("employment_status")[num_col].mean()
    train[f"{num_col}_mean_by_employment"] = train["employment_status"].map(group_mean)
    test[f"{num_col}_mean_by_employment"] = test["employment_status"].map(group_mean)

    train[f"{num_col}_diff_from_employment"] = (
        train[num_col] - train[f"{num_col}_mean_by_employment"]
    )
    test[f"{num_col}_diff_from_employment"] = test[num_col] - test[f"{num_col}_mean_by_employment"]

    train[f"{num_col}_pct_from_employment"] = train[f"{num_col}_diff_from_employment"] / (
        train[f"{num_col}_mean_by_employment"] + 1
    )
    test[f"{num_col}_pct_from_employment"] = test[f"{num_col}_diff_from_employment"] / (
        test[f"{num_col}_mean_by_employment"] + 1
    )

print("   - Созданы среднее, отклонение и % отклонение для каждого числового признака")

# Комбинации employment с категориями
for cat_col in ["education_level", "marital_status", "loan_purpose"]:
    name = f"employment_{cat_col}"
    train[name] = train["employment_status"].astype(str) + "_" + train[cat_col].astype(str)
    test[name] = test["employment_status"].astype(str) + "_" + test[cat_col].astype(str)
print("   - Комбинации employment с education, marital_status, loan_purpose")

# ПРИЗНАК 4: Группировки по категориям
print("\nПРИЗНАК 4: Групповая статистика по категориям")
print("   Обоснование: стандартная практика для извлечения паттернов")

# По grade_subgrade (сильная связь: 60-95%)
for num_col in NUMS:
    group_mean = train.groupby("grade_subgrade")[num_col].mean()
    group_std = train.groupby("grade_subgrade")[num_col].std()

    train[f"{num_col}_mean_by_grade"] = train["grade_subgrade"].map(group_mean)
    test[f"{num_col}_mean_by_grade"] = test["grade_subgrade"].map(group_mean)

    train[f"{num_col}_std_by_grade"] = train["grade_subgrade"].map(group_std)
    test[f"{num_col}_std_by_grade"] = test["grade_subgrade"].map(group_std)

    train[f"{num_col}_diff_from_grade"] = train[num_col] - train[f"{num_col}_mean_by_grade"]
    test[f"{num_col}_diff_from_grade"] = test[num_col] - test[f"{num_col}_mean_by_grade"]

print("   - Статистика по grade_subgrade (A-F): mean, std, diff")

# По loan_purpose (77-82%)
for num_col in ["annual_income", "credit_score", "debt_to_income_ratio"]:
    group_mean = train.groupby("loan_purpose")[num_col].mean()
    train[f"{num_col}_mean_by_purpose"] = train["loan_purpose"].map(group_mean)
    test[f"{num_col}_mean_by_purpose"] = test["loan_purpose"].map(group_mean)

    train[f"{num_col}_diff_from_purpose"] = train[num_col] - train[f"{num_col}_mean_by_purpose"]
    test[f"{num_col}_diff_from_purpose"] = test[num_col] - test[f"{num_col}_mean_by_purpose"]

print("- Статистика по loan_purpose")

# По education_level (78-83%)
for num_col in ["annual_income", "credit_score"]:
    group_mean = train.groupby("education_level")[num_col].mean()
    train[f"{num_col}_mean_by_education"] = train["education_level"].map(group_mean)
    test[f"{num_col}_mean_by_education"] = test["education_level"].map(group_mean)

print("- Статистика по education_level")

# ПРИЗНАК 5: Комбинации категорий
print("\nПРИЗНАК 5: Комбинации категориальных признаков")

important_cat_pairs = [
    ("employment_status", "education_level"),
    ("employment_status", "marital_status"),
    ("grade_subgrade", "loan_purpose"),
    ("education_level", "loan_purpose"),
]

for col1, col2 in important_cat_pairs:
    name = f"{col1}_{col2}_combo"
    train[name] = train[col1].astype(str) + "_" + train[col2].astype(str)
    test[name] = test[col1].astype(str) + "_" + test[col2].astype(str)

print(f"- Создано {len(important_cat_pairs)} комбинаций")


# ПРИЗНАК 6: Флаги
print("\nПРИЗНАК 6: Бинарные флаги")

train["high_credit_score"] = (train["credit_score"] >= 720).astype("int8")
test["high_credit_score"] = (test["credit_score"] >= 720).astype("int8")

train["low_debt_ratio"] = (train["debt_to_income_ratio"] <= 0.1).astype("int8")
test["low_debt_ratio"] = (test["debt_to_income_ratio"] <= 0.1).astype("int8")

train["high_income"] = (train["annual_income"] >= 60000).astype("int8")
test["high_income"] = (test["annual_income"] >= 60000).astype("int8")

train["small_loan"] = (train["loan_amount"] <= 10000).astype("int8")
test["small_loan"] = (test["loan_amount"] <= 10000).astype("int8")

train["low_interest"] = (train["interest_rate"] <= 11).astype("int8")
test["low_interest"] = (test["interest_rate"] <= 11).astype("int8")

print("\nFeature Engineering завершён!")
print("Было признаков: 11")
print(f"Стало признаков: {train.shape[1]}")
print(f"Создано новых: {train.shape[1] - 11}")

# Сохраняем для последующего использования
train_fe = train.copy()
test_fe = test.copy()

# LightAutoML (baseline)

In [ ]:
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task

# Подготовка данных для AutoML
X_train_automl = pd.read_csv("train.csv")
X_test_automl = pd.read_csv("test.csv")

test_ids = X_test_automl["id"].copy()
target = X_train_automl["loan_paid_back"].copy()

# Удаляем служебные колонки
train = X_train_automl.drop(columns=["id", "loan_paid_back"])
test = X_test_automl.drop(columns=["id"])

# AutoML работает лучше с исходными данными
X_train_automl = X_train_automl.assign(loan_paid_back=target)

print("\nОбучение LightAutoML")

task = Task("binary")
automl = TabularAutoML(
    task=task,
    timeout=3600,
    cpu_limit=4,
    reader_params={
        "n_jobs": 4,
        "cv": 5,
        "random_state": 42,
    },
)

# Обучение
oof_pred_automl = automl.fit_predict(X_train_automl, roles={"target": "loan_paid_back"})

# Предсказание
test_pred_automl = automl.predict(X_test_automl)
pred_proba_automl = test_pred_automl.data.flatten()

from sklearn.metrics import roc_auc_score

automl_cv_auc = roc_auc_score(target, oof_pred_automl.data.flatten())

print(f"\nLightAutoML CV AUC: {automl_cv_auc:.5f}")

# Сохранение submission
submission = pd.DataFrame({"id": test_ids, "loan_paid_back": pred_proba_automl})

submission.to_csv("submission_automl.csv", index=False)

print("\nФайл submission.csv успешно сохранён!")
print(submission.head())

- Private Score 0.92584
- Public Score 0.92477

# Catboost pipeline

## Почему Pipeline избыточен

1. **CatBoost работает с Pool объектами** - это специальная структура данных CatBoost, которая инкапсулирует данные и категориальные признаки. Pipeline не умеет работать с Pool

2. **Двухэтапный процесс (Optuna + CV)** - сначала идёт оптимизация на hold-out, потом обучение с CV. Pipeline не добавит ценности ни на одном из этапов

3. **GPU-специфичные параметры** - `task_type`, `devices` передаются напрямую в конструктор CatBoost, Pipeline только усложнит их управление

In [ ]:
import warnings

import optuna
import pandas as pd
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, train_test_split

warnings.filterwarnings("ignore")

# Данные
train_data = train_fe.copy()
test_data = test_fe.copy()
y = target.copy().reset_index(drop=True)
test_ids = test_ids.copy()

# Категориальные признаки
cat_cols = train_data.select_dtypes(include=["object", "category"]).columns.tolist()
print(f"Категориальные признаки: {cat_cols}")

# Индексы категориальных колонок
cat_feature_indices = [train_data.columns.get_loc(c) for c in cat_cols]

# для воспроизводимости
RND = 42


# функция Optuna
def objective(trial):
    # Только поддерживаемые GPU лоссы
    loss = trial.suggest_categorical("loss_function", ["Logloss", "CrossEntropy"])

    params = {
        "iterations": trial.suggest_int("iterations", 200, 2000),
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.001, 0.15),
        "depth": trial.suggest_int("depth", 1, 8),
        "l2_leaf_reg": trial.suggest_loguniform("l2_leaf_reg", 0.1, 10.0),
        "bagging_temperature": trial.suggest_uniform("bagging_temperature", 0.0, 1.0),
        # GPU
        "task_type": "GPU",
        "devices": "0",
        "random_seed": RND,
        "loss_function": loss,
        "eval_metric": "AUC",
        "verbose": False,
    }

    # Hold-out
    X_tr, X_val, y_tr, y_val = train_test_split(
        train_data, y, test_size=0.2, stratify=y, random_state=RND
    )

    train_pool = Pool(X_tr, label=y_tr, cat_features=cat_feature_indices)
    val_pool = Pool(X_val, label=y_val, cat_features=cat_feature_indices)

    model = CatBoostClassifier(**params)

    model.fit(
        train_pool, eval_set=val_pool, use_best_model=True, early_stopping_rounds=50, verbose=False
    )

    pred = model.predict_proba(X_val)[:, 1]
    auc = roc_auc_score(y_val, pred)

    del model, X_tr, X_val, y_tr, y_val, train_pool, val_pool
    gc.collect()

    return auc


# Оптимизация через optuna

study = optuna.create_study(direction="maximize", study_name="catboost_gpu_opt")
print("\nЗапуск Optuna (GPU)")
study.optimize(objective, n_trials=30, n_jobs=1)

print("\nOptuna завершён")
print(f"Лучший AUC (hold-out): {study.best_value:.5f}")
print("Лучшие параметры:")
print(study.best_params)

best_params = study.best_params.copy()

# GPU-настройки для финального обучения
best_params.update(
    {"task_type": "GPU", "devices": "0", "random_seed": RND, "verbose": 200, "eval_metric": "AUC"}
)


N_FOLDS = 3
skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=RND)

oof_cat = np.zeros(len(train_data))
pred_cat = np.zeros(len(test_data))

for fold, (tr_idx, val_idx) in enumerate(skf.split(train_data, y)):
    print(f"\n=== Fold {fold + 1}/{N_FOLDS} ===")

    X_tr = train_data.iloc[tr_idx].reset_index(drop=True)
    X_val = train_data.iloc[val_idx].reset_index(drop=True)
    y_tr = y.iloc[tr_idx].reset_index(drop=True)
    y_val = y.iloc[val_idx].reset_index(drop=True)

    train_pool = Pool(X_tr, label=y_tr, cat_features=cat_feature_indices)
    val_pool = Pool(X_val, label=y_val, cat_features=cat_feature_indices)
    test_pool = Pool(test_data, cat_features=cat_feature_indices)

    model = CatBoostClassifier(**best_params)

    model.fit(
        train_pool, eval_set=val_pool, use_best_model=True, early_stopping_rounds=50, verbose=200
    )

    oof_cat[val_idx] = model.predict_proba(X_val)[:, 1]
    pred_cat += model.predict_proba(test_data)[:, 1] / N_FOLDS

    fold_auc = roc_auc_score(y_val, oof_cat[val_idx])
    print(f"Fold AUC: {fold_auc:.5f}")

    del X_tr, X_val, y_tr, y_val, train_pool, val_pool, test_pool, model
    gc.collect()

# Полный CV AUC
final_auc = roc_auc_score(y, oof_cat)
print(f"\nИтоговый CV AUC CatBoost: {final_auc:.5f}")


# Сохранение submission
submission = pd.DataFrame({"id": test_ids, "loan_paid_back": pred_cat})

submission.to_csv("submission_catboost_gpu_optuna.csv", index=False)
print("\nSubmission сохранён: submission_catboost_gpu_optuna.csv")

- Private Score: 0.92156
- Public Score: 0.92087

# CatBoost + LightGBM

## Почему Pipeline избыточен 

1. **StackingClassifier уже является своего рода pipeline** - он сам управляет потоком данных через базовые модели к мета-модели

2. **Категориальные признаки обрабатываются напрямую** - CatBoost и LightGBM работают с категориями нативно через параметры `cat_features` и `categorical_feature`, которые передаются в конструктор моделей

3. **Нет препроцессинга, который нужно применять последовательно** - данные уже прошли feature engineering (`train_fe`, `test_fe`), остаётся только преобразование типов в `category`, что делается один раз

4. **Pipeline не упростит код** - пришлось бы создавать обёртки для передачи `cat_features` и `categorical_feature`, что только усложнит структуру

5. **StackingClassifier + cross_val_predict уже обеспечивают всю нужную логику** - управление CV-фолдами, OOF предсказаниями и финальным обучением

In [ ]:
import gc
import warnings

import lightgbm as lgb
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_predict

warnings.filterwarnings("ignore")

train_data = train_fe.copy()
test_data = test_fe.copy()
y = target.copy().reset_index(drop=True)
test_ids = test_ids.copy()

# Категориальные признаки
cat_cols = train_data.select_dtypes(include=["object", "category"]).columns.tolist()

# Привести object в category
for c in cat_cols:
    train_data[c] = train_data[c].astype("category")
    test_data[c] = test_data[c].astype("category")

cat_feature_indices = [train_data.columns.get_loc(c) for c in cat_cols]

print(f"Категориальных признаков: {len(cat_cols)}")
print(f"Размер train: {train_data.shape}, test: {test_data.shape}")

RND = 42

# CatBoost (GPU)
catboost_model = CatBoostClassifier(
    loss_function="Logloss",
    iterations=1500,
    learning_rate=0.08,
    depth=7,
    eval_metric="AUC",
    verbose=0,
    random_seed=RND,
    cat_features=cat_feature_indices,
    early_stopping_rounds=50,
    task_type="GPU",
    devices="0",
)

# LightGBM
lightgbm_model = lgb.LGBMClassifier(
    objective="binary",
    metric="auc",
    n_estimators=2000,
    learning_rate=0.05,
    verbosity=-1,
    random_state=RND,
    n_jobs=-1,
    categorical_feature=cat_cols,
    device_type="gpu",
    gpu_platform_id=0,
    gpu_device_id=0,
)

base_estimators = [
    ("catboost", catboost_model),
    ("lightgbm", lightgbm_model),
]

# Мета-модель
meta_model = LogisticRegression(
    C=1.0,
    random_state=RND,
    max_iter=1000,
    solver="lbfgs",
)

stacking_clf = StackingClassifier(
    estimators=base_estimators,
    final_estimator=meta_model,
    cv=3,
    stack_method="predict_proba",
    n_jobs=1,
    verbose=2,
    passthrough=False,
)

# Обучение стэкинга
stacking_clf.fit(train_data, y)

# Предсказание на test
final_pred = stacking_clf.predict_proba(test_data)[:, 1]

# Сохранение submission
submission = pd.DataFrame({"id": test_ids, "loan_paid_back": final_pred})
submission.to_csv("submission_sklearn_stacking_gpu.csv", index=False)
print("Saved: submission_sklearn_stacking_gpu.csv")

# Оценка качества через OOF предсказания
print("Оценка качества (OOF)")

oof_predictions = cross_val_predict(
    stacking_clf,
    train_data,
    y,
    cv=5,
    method="predict_proba",
    n_jobs=1,
    verbose=1,
)[:, 1]

oof_auc = roc_auc_score(y, oof_predictions)
print(f"OOF AUC: {oof_auc:.5f}")

gc.collect()

- Private score: 0.92184
- Public score: 0.92104

# LGBM

## Почему Pipeline избыточен

1. **Кодирование делается один раз на train+test вместе** - это нельзя поместить в Pipeline, так как Pipeline применяет трансформации отдельно на каждом фолде CV

2. **LightGBM принимает `categorical_feature` как параметр fit()** - Pipeline не умеет пробрасывать такие специфичные параметры через стандартный `.fit(X, y)`

3. **Для работы с Pipeline пришлось бы создавать обёртки-классы** - это усложняет код без реальной пользы

In [ ]:
import gc

import lightgbm as lgbm
import numpy as np
import optuna
import pandas as pd
from optuna.samplers import TPESampler
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

# Подготовка данных после FE
X = train_fe.copy()
y = target.copy().reset_index(drop=True)
test_final = test_fe.copy()

# Кодирование категориальных признаков для LightGBM
cat_cols = X.select_dtypes(include=["object"]).columns.tolist()
print(f"Найдено категориальных признаков: {len(cat_cols)}")
print(f"Категории: {cat_cols}")

# объединяем train + test
for col in cat_cols:
    # Собираем все уникальные значения из train и test
    all_values = pd.concat([X[col], test_final[col]]).astype(str).unique()
    # Создаём маппинг: строка -> целое число
    label_to_id = {v: i for i, v in enumerate(all_values)}
    # Применяем маппинг
    X[col] = X[col].astype(str).map(label_to_id).astype("int32")
    test_final[col] = test_final[col].astype(str).map(label_to_id).astype("int32")

# индексы категориальных признаков
cat_col_indices = [X.columns.get_loc(c) for c in cat_cols]


# Функция для оптимизации
def objective(trial):
    # Гиперпараметры для оптимизации
    params = {
        "objective": "binary",
        "metric": "auc",
        "verbosity": -1,
        "random_state": 42,
        "n_estimators": trial.suggest_int("n_estimators", 500, 5000, step=500),
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.05, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 8),
        "num_leaves": trial.suggest_int("num_leaves", 15, 128),
    }

    # CV
    num_folds = 3
    skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

    oof_preds = np.zeros(len(X))
    fold_scores = []

    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        model = lgbm.LGBMClassifier(**params)

        model.fit(
            X_train,
            y_train,
            eval_set=[(X_val, y_val)],
            categorical_feature=cat_col_indices,
            callbacks=[lgbm.early_stopping(stopping_rounds=100, verbose=False)],
        )

        val_pred = model.predict_proba(X_val)[:, 1]
        oof_preds[val_idx] = val_pred

        score = roc_auc_score(y_val, val_pred)
        fold_scores.append(score)

        del model, X_train, X_val, y_train, y_val
        gc.collect()

    cv_score = np.mean(fold_scores)

    # Логируем промежуточные результаты
    trial.set_user_attr("cv_std", np.std(fold_scores))
    trial.set_user_attr("fold_scores", fold_scores)

    return cv_score


study = optuna.create_study(
    direction="maximize", sampler=TPESampler(seed=42), study_name="lgbm_optimization"
)

# Оптимизация
study.optimize(objective, n_trials=5, show_progress_bar=True)

# Результаты оптимизации
print(f"Лучший AUC: {study.best_value:.5f}")
print("Лучшие параметры:")
for key, value in study.best_params.items():
    print(f"  {key}: {value}")

# CV std для лучшей модели
best_trial = study.best_trial
print(f"\nCV std: {best_trial.user_attrs['cv_std']:.5f}")
print(f"Fold scores: {[f'{s:.5f}' for s in best_trial.user_attrs['fold_scores']]}")

# Обучение финальной модели с лучшими параметрами
best_params = study.best_params.copy()
best_params.update(
    {
        "objective": "binary",
        "metric": "auc",
        "verbosity": -1,
        "random_state": 42,
    }
)

num_folds = 3
skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

oof_preds = np.zeros(len(X))
test_preds = np.zeros((len(test_final), num_folds))
scores = []


for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    print(f"\n--- Fold {fold + 1}/{num_folds} ---")

    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    model = lgbm.LGBMClassifier(**best_params)

    model.fit(
        X_train,
        y_train,
        eval_set=[(X_val, y_val)],
        categorical_feature=cat_col_indices,
        callbacks=[
            lgbm.log_evaluation(500),
            lgbm.early_stopping(stopping_rounds=100, verbose=False),
        ],
    )

    # валидационные предсказания
    val_pred = model.predict_proba(X_val)[:, 1]
    oof_preds[val_idx] = val_pred

    # Тестовые предсказания
    test_pred = model.predict_proba(test_final)[:, 1]
    test_preds[:, fold] = test_pred

    score = roc_auc_score(y_val, val_pred)
    scores.append(score)
    print(f"Fold {fold + 1} AUC: {score:.5f}")

    del model, X_train, X_val, y_train, y_val
    gc.collect()


# Итоги
cv_mean = np.mean(scores)
cv_std = np.std(scores)
oof_auc = roc_auc_score(y, oof_preds)

print(f"CV AUC: {cv_mean:.5f} ± {cv_std:.5f}")
print(f"OOF AUC: {oof_auc:.5f}")

# Submission
submission = pd.DataFrame({"id": test_ids, "loan_paid_back": test_preds.mean(axis=1)})
submission.to_csv("submission_lightgbm_optuna.csv", index=False)
print("Submission сохранён как 'submission_lightgbm_optuna.csv'")

- Public Score: 0.91943
- Private Score: 0.91998